# Poincare Map

In [1]:
import rebound
import numpy as np
import math
import ctypes

In [2]:
def ydot(x,y,vx,Cj,mu1,mu2):
    r1 = math.sqrt((x+mu2)*(x+mu2) + y*y)
    r2 = math.sqrt((x-mu1)*(x-mu1) + y*y)
    vy = np.sqrt(-Cj + (x*x+y*y) + 2.*(mu1/r1 + mu2/r2) - vx*vx)
    return vy
def hyper(sim):
    ps = sim.particles
    dx  = ps[1].x -ps[0].x
    dy  = ps[1].y -ps[0].y
    dvx = ps[1].vx-ps[0].vx
    dvy = ps[1].vy-ps[0].vy
    return dx*dvx + dy*dvy
def mod2pi(x):
    if x>np.pi:
        return mod2pi(x-2.*np.pi)
    if x<-np.pi:
        return mod2pi(x+2.*np.pi)
    return x

In [39]:
N_points_max = 10000
N_points = 0
poincare_map = np.zeros((N_points_max,2))

while N_points <N_points_max:
    print "new sim"
    sim = rebound.Simulation()
    sim.integrator = "ias15"
    sim.add(m=1.)
    
    a = np.random.uniform(0.9*0.62996052,1.1*0.62996052)
    print a
    sim.add(m=0, a=a, e=0.05)
    sim.add(m=1e-4, a=1., e=0.001)
    sim.move_to_com()
    dt = 0.13
    dt_epsilon = 0.0001
    sign = hyper(sim)
    while sim.t<50000. and N_points < N_points_max:
        oldt = sim.t
        olddt = sim.dt
        sim.integrate(oldt+dt)
        nsign = hyper(sim)
        if sign*nsign < 0.:
            leftt = oldt
            rightt = sim.t
            sim.dt = -olddt
            while (rightt-leftt > dt_epsilon):
                midt = (leftt+rightt)/2.
                rebound.clibrebound.reb_integrator_reset(ctypes.byref(sim))
                sim.integrate(midt, exact_finish_time=1)
                msign = hyper(sim)
                if msign*sign > 0.:
                    leftt = midt
                    sim.dt = 0.3*olddt
                else:
                    rightt = midt
                    sim.dt = -0.3*olddt
            o = sim.calculate_orbits()
            if mod2pi(o[0].f)<0.3:
                tp = np.pi*2.
                phi = o[0].l-2.*o[1].l+o[0].omega+o[0].Omega
                phid = tp/o[0].P-2.*tp/o[1].P
                poincare_map[N_points] = [mod2pi(phi),phid]
                N_points += 1
            sim.dt = olddt
            sim.integrate(oldt+dt)
        sign = nsign

new sim
0.588343889316


In [3]:
%matplotlib inline
pm_x = poincare_map[:,0]
pm_y = poincare_map[:,1]
if 0:
    pm_maskedx = np.ma.masked_where(pm_x>0., pm_x)
    pm_maskedy = np.ma.masked_where(pm_x>0., pm_y)
else:
    pm_maskedx = pm_x
    pm_maskedy = pm_y

    
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(111)
ax.scatter(pm_maskedx,pm_maskedy,marker=".",s=1.,alpha=.81)
print sim.t

NameError: name 'poincare_map' is not defined

In [4]:
sim = rebound.Simulation()
sim.integrator = "ias15"
sim.add(m=1.)

a = 0.588343889316
a = 0.58
sim.add(m=0, a=a, e=0.05)
#sim.add(m=1e-4, a=1., e=0.001)
sim.move_to_com()
sim.status()
Nplot = 1000
res = np.zeros((Nplot, 2))
times = np.linspace(0.,18845.19,Nplot)
for i, t in enumerate(times):
    sim.integrate(t+0.0000,exact_finish_time=1)
    #o = sim.calculate_orbits()
    #phi = o[0].l-2.*o[1].l+o[0].omega+o[0].Omega
    #res[i] = [t,mod2pi(phi)]
    res[i] = [t,sim.particles[0].x]

---------------------------------
REBOUND version:     	2.6.0
REBOUND built on:    	Aug 20 2015 11:50:27
Number of particles: 	2
Selected integrator: 	ias15
Simulation time:     	0.000000
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, id=-1 m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, id=-1 m=0.0 x=0.551 y=0.0 z=0.0 vx=0.0 vy=1.38044418007 vz=0.0>
---------------------------------


In [5]:
print res

[[     0.              0.        ]
 [    18.86405405      0.        ]
 [    37.72810811      0.        ]
 ..., 
 [ 18807.46189189      0.        ]
 [ 18826.32594595      0.        ]
 [ 18845.19            0.        ]]


In [11]:
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(111)
ax.scatter(res[:,0], res[:,1],marker=".",s=1.)

NameError: name 'plt' is not defined